In [1]:
# 必要なライブラリを読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
#trainデータをデータフレームの形で読み込んでtrainという名前の変数に代入
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train = train.drop(train[train['1stFlrSF'] >= 4500].index)
train = train.drop(train[train['GrLivArea'] >= 4100].index)
train = train.drop(train[train['GarageCars'] >= 4].index)
train = train.drop(train[train['GarageArea'] >= 1200].index)
train = train.drop(train[train['TotalBsmtSF'] >= 6000].index)
train = train.drop(train[train['2ndFlrSF'] >= 1850].index)

In [4]:
train['AllFlrSF'] = train['TotalBsmtSF'] + train['1stFlrSF'] + train['2ndFlrSF'] 

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import seaborn as sns

# X = train.loc[:, ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea','AllFlrSF','FullBath','TotRmsAbvGrd','YearBuilt','YearRemodAdd']].values
y = train.SalePrice
X=train
#データの標準化
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_std = ss.fit_transform(X)

# ホールド・アウト法による分割
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.3, random_state=0)

#ランダムフォレストによる学習(回帰）
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=80, max_features='auto')
rf.fit(X_train, y_train)
print('Training done using Random Forest')

ranking = np.argsort(-rf.feature_importances_)
f, ax = plt.subplots(figsize=(11, 9))
sns.barplot(x=rf.feature_importances_[ranking], y=X_train.columns.values[ranking], orient='h')
ax.set_xlabel("feature importance")
plt.tight_layout()
plt.show()
# lr = RandomForestRegressor(random_state=0, n_estimators=10)
# lr.fit(X_std, y)

# importances = lr.feature_importances_
# # 特徴量の重要度をグラフに出力
# n_features = len(train.feature_names)
# plt.figure(figsize=(12, 8))
# plt.barh(range(n_features), lr.feature_importances_ , align='center')
# plt.yticks(np.arange(n_features), lr.feature_names)
# plt.show()

# # RMSEの出力
# print('train RMSE:%.3f'%(mse(y_train, lr.predict(X_train)) ** (1/2)))
# print('test RMSE:%.3f'%(mse(y_test, lr.predict(X_test)) ** (1/2)))

ValueError: could not convert string to float: 'Normal'

In [6]:
test['AllFlrSF'] = test['TotalBsmtSF'] + test['1stFlrSF'] + test['2ndFlrSF'] 
test['AllFlrSF'].isnull().sum()

1

In [7]:
test['GarageCars'] = test['GarageCars'].fillna(test['GarageCars'].median())
test['GarageArea'] = test['GarageArea'].fillna(test['GarageArea'].median())
test['AllFlrSF'] = test['AllFlrSF'].fillna(test['AllFlrSF'].median())
print(test['GrLivArea'].isnull().sum())
print(test['GarageCars'].isnull().sum())
print(test['GarageArea'].isnull().sum())
print(test['FullBath'].isnull().sum())

0
0
0
0


In [8]:
X_pred = test.loc[:, ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea','AllFlrSF','FullBath','TotRmsAbvGrd','YearBuilt','YearRemodAdd']].values
ss = StandardScaler()
X_pred = ss.fit_transform(X_pred)
# 予測
y_pred = lr.predict(X_pred)

# 提出csvファイルの作成
submit_file = pd.DataFrame({'Id' : test['Id'], 'SalePrice' : y_pred})
submit_file.to_csv('submit_1.csv', index = False)